In [ ]:
!pip install keras_metrics

In [ ]:
import re
from tqdm import tqdm

import keras_metrics
import numpy as np
import pandas as pd
from tensorflow import keras
from tensorflow.keras import regularizers 




In [ ]:
df = pd.read_csv('pasangan_baik.csv')

In [ ]:
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open('min-ind.txt', "r", encoding="utf-8") as f:
 
  lines = f.read().split("\n")



  #print('lines',lines)
#print('lines',lines[:len(lines)])

print(len(lines))
for line in lines:
  if len(line.split(','))==2:
    input_text, target_text = line.split(",")
  # We use "tab" as the "start sequence" character
  # for the targets, and "\n" as "end sequence" character.
  target_text = "\t" + target_text + "\n"
  input_texts.append(input_text)
  target_texts.append(target_text)
  for char in input_text:
      if char not in input_characters:
          input_characters.add(char)
  for char in target_text:
      if char not in target_characters:
          target_characters.add(char)

13762


In [ ]:
#Buat kamus dan update input dan target teks
def append_value(dict_obj, key, value):
    if key in dict_obj:
        if not isinstance(dict_obj[key], list):
            dict_obj[key] = [dict_obj[key]]
        dict_obj[key].append(value)
    else:
        dict_obj[key] = value

df = pd.DataFrame([input_texts, target_texts], index=['input', 'target']).T.drop_duplicates()
clean_tab_n = lambda x: re.sub(r'(\n)+$','',re.sub(r'^(\t)+', '', x))
df['target'] = df['target'].apply(clean_tab_n)

#update input_txt dan target-text
input_texts = df['input'].to_list()
target_texts = df['target'].to_list()

kamus = {}

for i, row in tqdm(df.iterrows()):
  key = row['input']
  append_value(kamus, key, row['target'])

13208it [00:01, 10599.31it/s]


In [ ]:
kamus

{'nan': ['yang', 'maadoki apo sajo"'],
 'di': 'di',
 'dari': 'dari',
 'jo': 'dengan',
 'dan': 'dan',
 'ko': 'ini',
 'adolah': 'adalah',
 'langau': 'nyamuk',
 'pado': 'pada',
 'juo': 'juga',
 'bagian': ['bagian', 'bidang', 'divisi'],
 'marupokan': 'merupakan',
 'dalam': 'dalam',
 'indonesia': 'indonesia',
 'biasonyo': 'biasanya',
 'spesies': 'spesies',
 'untuak': 'untuk',
 'kelas': 'kelas',
 'famili': 'famili',
 'ordo': 'ordo',
 'filum': 'filum',
 'arthropoda': 'arthropoda',
 'insecta': 'insecta',
 'daun': 'daun',
 'larva': 'larva',
 'diptera': 'diptera',
 'mamakan': 'memakan',
 'herbivora': 'herbivora',
 'dek': 'karena',
 'sabagai': 'sebagai',
 'ka': 'ka',
 'kota': 'kota',
 'taun': 'tahun',
 'ado': 'ada',
 'iko': 'ini',
 'inyo': 'dia',
 'manjadi': 'menjadi',
 'sabuah': 'sebuah',
 'atau': 'atau',
 'kabupaten': 'kabupaten',
 'marupoan': 'merupakan',
 'urang': ['orang', 'kepala', 'tenaga'],
 'provinsi': 'provinsi',
 'sarato': 'serta',
 'kecamatan': 'kecamatan',
 'barat': 'barat',
 'indak'

In [ ]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

In [ ]:
print("Number of samples:", len(input_texts))
print("Number of unique input tokens:", num_encoder_tokens)
print("Number of unique output tokens:", num_decoder_tokens)
print("Max sequence length for inputs:", max_encoder_seq_length)
print("Max sequence length for outputs:", max_decoder_seq_length)

Number of samples: 13208
Number of unique input tokens: 35
Number of unique output tokens: 32
Max sequence length for inputs: 21
Max sequence length for outputs: 17


In [ ]:
input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])
 
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype="float32"
)
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.0
    decoder_input_data[i, t + 1 :, input_token_index[" "]] = 1.0
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.0
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
    decoder_input_data[i, t + 1 :, target_token_index[" "]] = 1.0
    decoder_target_data[i, t:, target_token_index[" "]] = 1.0

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
seed = 7
np.random.seed(seed)
kfold_index,test_index = train_test_split(list(range(len(encoder_input_data))),test_size=0.2,random_state=seed)
fold_var = 5
kf = KFold(n_splits = fold_var,random_state=seed, shuffle=True)
validation_accuracy = []
validation_loss = []
exac_match_accuracy = []

In [ ]:
encoder_input_data_k = pd.Series(list(encoder_input_data)).iloc[kfold_index].to_numpy()
decoder_input_data_k = pd.Series(list(decoder_input_data)).iloc[kfold_index].to_numpy()
decoder_target_data_k = pd.Series(list(decoder_target_data)).iloc[kfold_index].to_numpy()

encoder_input_data_k = np.array([list(en) for en in encoder_input_data_k])
decoder_input_data_k = np.array([list(en) for en in decoder_input_data_k])
decoder_target_data_k = np.array([list(en) for en in decoder_target_data_k])

encoder_input_data_test = pd.Series(list(encoder_input_data)).iloc[test_index].to_numpy()
decoder_input_data_test = pd.Series(list(decoder_input_data)).iloc[test_index].to_numpy()
decoder_target_data_test = pd.Series(list(decoder_target_data)).iloc[test_index].to_numpy()

encoder_input_data_test = np.array([list(en) for en in encoder_input_data_test])
decoder_input_data_test = np.array([list(en) for en in decoder_input_data_test])
decoder_target_data_test = np.array([list(en) for en in decoder_target_data_test])

In [ ]:
batch_size = 64  # Batch size for training.
epochs = 5  # Number of epochs to train for.
latent_dim_1 = 128
latent_dim_2 = 256
latent_dim_3 = 512 # Latent dimensionality of the encoding space.
num_samples = len(encoder_input_data)  # Number of samples to train on.

In [ ]:
for i,(train_index,val_index) in enumerate(kf.split(np.zeros(len(encoder_input_data_k)))):
  print('Iteration number : ', i+1)

  encoder_input_data_t = pd.Series(list(encoder_input_data_k)).iloc[train_index].to_numpy()
  decoder_input_data_t = pd.Series(list(decoder_input_data_k)).iloc[train_index].to_numpy()
  decoder_target_data_t = pd.Series(list(decoder_target_data_k)).iloc[train_index].to_numpy()

  encoder_input_data_val = pd.Series(list(encoder_input_data_k)).iloc[val_index].to_numpy()
  decoder_input_data_val = pd.Series(list(decoder_input_data_k)).iloc[val_index].to_numpy()
  decoder_target_data_val = pd.Series(list(decoder_target_data_k)).iloc[val_index].to_numpy()

  encoder_input_data_t = np.array([list(en) for en in encoder_input_data_t])
  decoder_input_data_t = np.array([list(en) for en in decoder_input_data_t])
  decoder_target_data_t = np.array([list(en) for en in decoder_target_data_t])

  encoder_input_data_val = np.array([list(en) for en in encoder_input_data_val])
  decoder_input_data_val = np.array([list(en) for en in decoder_input_data_val])
  decoder_target_data_val = np.array([list(en) for en in decoder_target_data_val])

  x_train = [encoder_input_data_t, decoder_input_data_t]
  y_train = decoder_target_data_t

  x_val = [encoder_input_data_val,decoder_input_data_val]
  y_val = decoder_target_data_val

  encoder_inputs = keras.Input(shape=(None,num_encoder_tokens))
  encoder_outputs,forward_h,forward_c,backward_h,backward_c=keras.layers.Bidirectional(keras.layers.LSTM(latent_dim_2, return_state=True, kernel_regularizer=regularizers.l2(l2=1e-5),
    bias_regularizer=regularizers.l2(1e-5)),merge_mode='concat',name='encoder_lstm1')(encoder_inputs)

  # We discard `encoder_outputs` and only keep the states.
  state_h = keras.layers.Concatenate()([forward_h,forward_c])
  state_c = keras.layers.Concatenate()([backward_h,backward_c])
  encoder_states = [state_h,state_c] # hidden and cell state 

  # Set up the decoder, using `encoder_states` as initial state.
  decoder_inputs = keras.Input(shape=(None, num_decoder_tokens))
  # We set up our decoder to return full output sequences,
  # and to return internal states as well. We don't use the
  # return states in the training model, but we will use them in inference.
  decoder_outputs,*decoder_states= keras.layers.LSTM(latent_dim_3, 
                                                    return_sequences=True, 
                                                    return_state=True, kernel_regularizer=regularizers.l2(l2=1e-5),
    bias_regularizer=regularizers.l2(1e-5))(decoder_inputs, initial_state=encoder_states)

  decoder_dense = keras.layers.Dense(num_decoder_tokens, activation="softmax")
  #decoder_dropout = keras.layers.Dropout(0.8)
  decoder_outputs = decoder_dense(decoder_outputs)
  #decoder_outputs = decoder_dropout(decoder_outputs)
  # Define the model that will turn
  # `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
  model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

  # Compile Model
  model.compile(
      optimizer="adam", loss="categorical_crossentropy", 
      metrics=["accuracy",keras.metrics.Recall(),keras.metrics.Precision()]
  )
  # Fitting Model
  model_history = model.fit(
      x_train, y_train,
      batch_size=batch_size,
      epochs=epochs,
      validation_data = (x_val,y_val)
  )

  # Save model
  model_name = 'model_bilstm_256_{}-Fold-{}'.format(fold_var,i+1)
  model.save(model_name)

  # Define sampling models
  # Restore the model and construct the encoder and decoder.
  model_inf = keras.models.load_model(model_name)
  encoder_inputs = model_inf.input[0]  # input_1
  encoder_outputs,state_h_enc,state_c_enc,backward_h,backward_c  = model_inf.layers[1].output  # lstm_1
  states_h = keras.layers.Concatenate()([state_h_enc,state_c_enc])
  states_c = keras.layers.Concatenate()([backward_h,backward_c])
  encoder_states = [states_h,states_c]
  encoder_model = keras.Model(encoder_inputs, encoder_states)

  decoder_inputs = model_inf.input[1]  # input_2
  decoder_state_input_h = keras.Input(shape=(latent_dim_3,), name="input_3")
  decoder_state_input_c = keras.Input(shape=(latent_dim_3,), name="input_5")
  decoder_states_inputs = [decoder_state_input_h,decoder_state_input_c]
  decoder_lstm = model_inf.layers[5]
  decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(
      decoder_inputs, initial_state=decoder_states_inputs
  )
  decoder_states = [state_h_dec, state_c_dec]
  decoder_dense = model_inf.layers[6]
  decoder_outputs = decoder_dense(decoder_outputs)
  #decoder_dropout = model_inf.layers[7]
  #decoder_outputs = decoder_dropout(decoder_outputs)
  decoder_model = keras.Model(
      [decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states
  )
  # Reverse-lookup token index to decode sequences back to
  # something readable.
  reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
  reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())

  def decode_sequence(input_seq):
      # Encode the input as state vectors.
      states_value = encoder_model.predict(input_seq)

      # Generate empty target sequence of length 1.
      target_seq = np.zeros((1, 1, num_decoder_tokens))
      # Populate the first character of target sequence with the start character.
      target_seq[0, 0, target_token_index["\t"]] = 1.0

      # Sampling loop for a batch of sequences
      # (to simplify, here we assume a batch of size 1).
      stop_condition = False
      decoded_sentence = ""
      while not stop_condition:
          output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

          # Sample a token

          sampled_token_index = np.argmax(output_tokens[0, -1, :])
          sampled_char = reverse_target_char_index[sampled_token_index]
          decoded_sentence += sampled_char

          # Exit condition: either hit max length
          # or find stop character.
          if sampled_char == "\n" or len(decoded_sentence) > max_decoder_seq_length:
              stop_condition = True

          # Update the target sequence (of length 1).
          target_seq = np.zeros((1, 1, num_decoder_tokens))
          target_seq[0, 0, sampled_token_index] = 1.0

          # Update states
          states_value = [h, c]
      return decoded_sentence

  # Evaluate Model
  results = model.evaluate(x_val, y_val, batch_size=batch_size)
  results = dict(zip(model.metrics_names,results))
  
  # Save Evaluation Result
  validation_accuracy.append(results['accuracy'])
  validation_loss.append(results['loss'])
  
 
  # Check Result Translation
  
  # Check Result Translation
  
  list_match = []
  count = 0
  print("Checking Translation Accuracy")
  for seq_index in val_index:
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index : seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    decoded_sentence = decoded_sentence[:-1]
    # print("-",seq_index)
    #print("Input sentence:", ''.join(input_texts[seq_index]))
    #print("Decoded sentence:", ''.join(decoded_sentence))
    #print("Target sentence:", ''.join(target_texts[seq_index][1:-1]))
    exact_result = decoded_sentence==target_texts[seq_index][1:-1]
    #print("Exact Match:", ''.join(str(exact_result)))
    list_match.append(exact_result)
    count+=1
  print("Exact Match count : ", sum(list_match))
  print("Tested Data count : ", count)
  print("Translation Accuracy : ", sum(list_match)/count)
  

Iteration number :  1
Epoch 1/5
133/133 [==============================] - 61s 420ms/step - loss: 1.2290 - accuracy: 0.6753 - recall: 0.5807 - precision: 0.9524 - val_loss: 0.9745 - val_accuracy: 0.7330 - val_recall: 0.6050 - val_precision: 0.9827
Epoch 2/5
  6/133 [>.............................] - ETA: 49s - loss: 1.0151 - accuracy: 0.7160 - recall: 0.6166 - precision: 0.9556

KeyboardInterrupt: ignored

In [ ]:
# Show Accuracy Result
for i,val_acc in enumerate(validation_accuracy):
  print("Fold {} Accuracy : {}".format(i+1,val_acc))
print("Average Accuracy : %.2f%% (+/- %.2f%%)" % (np.mean(validation_accuracy), np.std(validation_accuracy)))

Fold 1 Accuracy : 0.9407671689987183
Fold 2 Accuracy : 0.9423704743385315
Fold 3 Accuracy : 0.9377596378326416
Fold 4 Accuracy : 0.9460934400558472
Fold 5 Accuracy : 0.945838451385498
Fold 6 Accuracy : 0.9392085075378418
Fold 7 Accuracy : 0.9400754570960999
Average Accuracy : 0.94% (+/- 0.00%)


In [ ]:
#Prep Test Data
x_test = [encoder_input_data_test,decoder_input_data_test]
y_test = decoder_target_data_test

# Evaluate Model
results = model.evaluate(x_test, y_test, batch_size=batch_size)
results = dict(zip(model.metrics_names,results))

54/54 [==============================] - 1s 19ms/step - loss: 0.2795 - accuracy: 0.9416 - recall_6: 0.9366 - precision_6: 0.9516


In [ ]:
# Check Result Translation
list_match = []
count = 0
print("Checking Translation Accuracy")
for seq_index in test_index:
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index : seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    decoded_sentence = decoded_sentence[:-1]
    print("-",seq_index)
    print("Input sentence:", ''.join(input_texts[seq_index]))
    print("Decoded sentence:", ''.join(decoded_sentence))
    print("Target sentence:", ''.join(target_texts[seq_index][1:-1]))
    exact_result = decoded_sentence==target_texts[seq_index][1:-1]
    print("Exact Match:", ''.join(str(exact_result)))
   
    list_match.append(exact_result)
    count+=1
print("Exact Match count : ", sum(list_match))
print("Tested Data count : ", count)
print("Translation Accuracy : ", sum(list_match)/count)

Streaming output truncated to the last 5000 lines.
Target sentence: melaut
Exact Match: False
- 12201
Input sentence: puak
Decoded sentence: puak
Target sentence: golongan
Exact Match: False
- 7996
Input sentence: mewakili
Decoded sentence: mewakili
Target sentence: mewakili
Exact Match: True
- 5310
Input sentence: mancanangkan
Decoded sentence: mencanangkan
Target sentence: mencanangkan
Exact Match: True
- 377
Input sentence: bahwa
Decoded sentence: bahwa
Target sentence: bahwa
Exact Match: True
- 852
Input sentence: saudi
Decoded sentence: saudi
Target sentence: saudi
Exact Match: True
- 11262
Input sentence: dipanggie
Decoded sentence: dipanggia
Target sentence: dipanggil
Exact Match: False
- 3028
Input sentence: asanyo
Decoded sentence: asanya
Target sentence: asalnya
Exact Match: False
- 2315
Input sentence: tokyo
Decoded sentence: tokya
Target sentence: tokyo
Exact Match: False
- 2471
Input sentence: kareno
Decoded sentence: karena
Target sentence: karena
Exact Match: True
- 4049

In [ ]:
!zip -r model_bilstm_256_5-Fold-5.zip model_bilstm_256_5-Fold-5

  adding: model_bilstm_256_5-Fold-5/ (stored 0%)
  adding: model_bilstm_256_5-Fold-5/assets/ (stored 0%)
  adding: model_bilstm_256_5-Fold-5/variables/ (stored 0%)
  adding: model_bilstm_256_5-Fold-5/variables/variables.index (deflated 67%)
  adding: model_bilstm_256_5-Fold-5/variables/variables.data-00000-of-00001 (deflated 6%)
  adding: model_bilstm_256_5-Fold-5/saved_model.pb (deflated 90%)
  adding: model_bilstm_256_5-Fold-5/keras_metadata.pb (deflated 91%)
